In [1]:
#imports
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from IPython.display import clear_output
import pandas as pd

In [2]:
#Classes and Functions

class Waiting: #various waiting functions depending on available information in html source
    #need to wait for html elements to fully load
    
    def waitID(self,Element):
        self.Element = Element
        try:
            element = WebDriverWait(driver,0).until(
                EC.presence_of_element_located((By.ID, Element))
            )
        finally:pass
        
    def waitNAME(self,Element):
        self.Element = Element
        try:
            element = WebDriverWait(driver,0).until(
                EC.presence_of_element_located((By.NAME, Element))
            )
        finally:pass
    
    def waitCSS(self,Element):
        self.Element = Element
        try:
            element = WebDriverWait(driver,0).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, Element))
            )
        finally:pass
        
    def waitCLASS(self,Element):
        self.Element = Element
        try:
            element = WebDriverWait(driver,0).until(
                EC.presence_of_element_located((By.CLASS_NAME, Element))
            )
        finally:pass
    
#input pathing information for final save        
a = r"C:\Users\Downloads\ "
SaveCSVLoc =input('Enter your desired folder path EX. -> ' + a)
SaveCSVNAME = input('Enter the name you would like to save your .CSV file as...')

Enter your desired folder path EX. -> C:\Users\Downloads\ C:\Users\Lhaye\Downloads\
Enter the name you would like to save your .CSV file as...Stock_Info


In [3]:
#Setup
driver = webdriver.Chrome()
BaseURL = 'https://www.google.com/finance/quote/' #to be used as a baseline for concatenation

Stock_List = ['SHOP:NYSE','AAPL:NASDAQ','TSLA:NASDAQ','OSPTX:INDEXTSI','T:NYSE','AC:TSE']

In [4]:
#data collection loop

n=0
WaitMessage = 'Waiting'
Stock_LT = [] #list of tuples for data collection - start empty

while n<len(Stock_List):
    driver.get(BaseURL + Stock_List[n]) #concat base url with selected stock address
    
    #wait command needed while information loads
    i=0
    w = 0
    while i == 0:
        try:
            Waiting().waitCLASS('AHmHk')
            StockPrice = driver.find_element(By.CLASS_NAME,'AHmHk').text
            i+=1
        except:
            print(WaitMessage + '.'*((w+4)%4)) #simple waiting 'animation' (loading dots with each iteration) - not necessary
            time.sleep(0.5) #on trial fail, wait then run again
            clear_output()
            w+=1
 
    Prev_Close = driver.find_element(By.CLASS_NAME,'P6K39c').text #copy previous close information (found in listed class)
    
    #Financials section only available on individual company stocks (ETFs/ industry portfolios not included)
    try:
        Revenue = driver.find_element(By.CLASS_NAME,'QXDnM').text #copy revenue info
    except:
        Revenue = 'Unavailable' #if not individual stock - no financial data available so set revenue variable to 'unavailable'

    
    Stock_LT.append((Stock_List[n],StockPrice,Prev_Close,Revenue)) # add tuple to list with each iteration
    n+=1
    
driver.quit()    #close chrome driver

In [5]:
df = pd.DataFrame(Stock_LT,columns=['Stock Ticker','Share Price','Previous Close','Revenue']) #set to dataframe object

In [6]:
df #show

,Stock Ticker,Share Price,Previous Close,Revenue
0,SHOP:NYSE,$38.89,$36.76,1.37B
1,AAPL:NASDAQ,$134.76,$133.41,90.15B
2,TSLA:NASDAQ,$122.40,$123.56,21.45B
3,OSPTX:INDEXTSI,"20,360.10","20,211.20",Unavailable
4,T:NYSE,$19.55,$19.41,30.04B
5,AC:TSE,$21.84,$21.91,5.32B


In [7]:
df.to_csv(SaveCSVLoc+SaveCSVNAME+'.csv',index=False,encoding='utf-8') #export file to .csv and save to computer